In [2]:
!wget https://raw.githubusercontent.com/nicolay-r/nlp-thirdgate/refs/heads/master/llm/transformers_llama.py

--2025-02-01 13:17:17--  https://raw.githubusercontent.com/nicolay-r/nlp-thirdgate/refs/heads/master/llm/transformers_llama.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 964 [text/plain]
Saving to: ‘transformers_llama.py.1’

transformers_llama. 100%[===================>]     964  --.-KB/s    in 0s      

2025-02-01 13:17:17 (71.7 MB/s) - ‘transformers_llama.py.1’ saved [964/964]



In [3]:
!pip install bulk-chain

In [1]:
from bulk_chain.core.utils import dynamic_init

llm = dynamic_init(class_dir=".",
                   class_filepath="transformers_llama.py",
                   class_name="Llama32")(model_name="deepseek-ai/DeepSeek-R1-Distill-Llama-8B",
                                         api_token="<YOUR-HF-API-TOKEN-GOES-HERE>",
                                         use_bf16=False,
                                         device="gpu",
                                         max_new_tokens=32768,
                                         temp=0.6)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0


# Setup Data and Inference Schema

In [10]:
def get_data():
    return [
        {"text": "What's the color of the sky? Explain why."},
    ]

schema = {
  "schema": [
    {"prompt": "{text}", "out": "response"}
  ]
}

# Usage

In [11]:
from bulk_chain.api import iter_content

for content in iter_content(get_data(), llm=llm, schema=schema):
    print(content)

[{'text': "What's the color of the sky? Explain why", 'response_prompt': "What's the color of the sky? Explain why", 'response': "<think>\nOkay, so I'm trying to figure out what color the sky is. I know that when I look up during the day, the sky appears blue, but sometimes it can look green or red, especially during sunrise or sunset. I'm a bit confused about why the sky is blue most of the time. Let me think through this step by step.\n\nFirst, I remember that the sky is mostly made up of air, which is a mixture of nitrogen and oxygen. Maybe the colors of these gases have something to do with it. I think when sunlight hits the Earth, it interacts with the atmosphere. I've heard the term Rayleigh scattering before. What does that mean exactly? I think it has to do with how light scatters in the atmosphere.\n\nSo, Rayleigh scattering is when light scatters off molecules in the air. Since nitrogen and oxygen are the main gases, maybe they are causing this scattering. But why does this m

# Time Measurement

In [12]:
import time

def measure_execution_time(data_it):
    start_time = time.perf_counter()

    total_len = 0
    for data in data_it:
        total_len = total_len + len(data)

    end_time = time.perf_counter()
    total_time = end_time - start_time

    return total_time, total_len

In [13]:
from bulk_chain.api import iter_content

def iter_llm_responses():
    for content in iter_content(get_data(), llm=llm, schema=schema):
        yield content[0]["response"]

l, t = measure_execution_time(data_it=iter_llm_responses())

print("Peformance: {}TPS".format(round(l/t, 2)))

Peformance: 0.18TPS
